In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install py_vncorenlp

In [ ]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git
%cd transformers
!pip3 install --upgrade .
%cd ..

In [ ]:
!pip install datasets

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip install lightning-transformers

In [ ]:
!pip install -U accelerate transformers

#PhoBert/BARTpho

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
import json
import pandas as pd
import csv
from datasets import load_dataset
from torch import nn
from transformers import RobertaForSequenceClassification, AutoTokenizer
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

In [ ]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='./')

In [ ]:

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')

text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

output = rdrsegmenter.word_segment(text)

print(output)
# ['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .', 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']

In [ ]:
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'

input_ids = torch.tensor([tokenizer.encode(sentence, padding = True, truncation=True)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

features

In [ ]:
def maplabel(ver):
  if ver == 'SUPPORTED':
    return 0
  elif ver == 'REFUTED':
    return 1
  else:
    return 2

In [ ]:
def maptarget(ver):
  if ver == 0:
    return 'SUPPORTED'
  elif ver == 1:
    return 'REFUTED'
  else:
    return 'NEI'

###PrepareData

In [ ]:
with open('/content/drive/MyDrive/DataUITDSC/ise-dsc01-train.json', 'r') as file:
  data = json.load(file)

len(data)

In [ ]:
for i in data:
  print(i)
  break

In [ ]:
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc 5.000 gia Hà Nội.\nBà Lan, vợ ông Chúc, cũng làm việc tại đây."

output = rdrsegmenter.word_segment(text)

print(output)

In [ ]:
"".join(rdrsegmenter.word_segment(data['7125']['context']))

In [ ]:
datafr = {'key': [], 'context': [], 'claim': [], 'label': [], 'evidence': []}
for key in tqdm(data):
  datafr['key'].append(key)
  datafr['context'].append("".join(rdrsegmenter.word_segment(data[key]['context'])))
  datafr['claim'].append(rdrsegmenter.word_segment(data[key]['claim'])[0].replace('\x00', ''))
  datafr['label'].append(maplabel(data[key]['verdict']))
  datafr['evidence'].append(data[key]['evidence'])


In [ ]:
train_df = pd.DataFrame(datafr)

train_df.head()

In [ ]:
train_df = train_df.fillna(' ')

In [ ]:
train_df.head()

In [ ]:
train_df.to_csv('/content/drive/MyDrive/dict_guid/DSC2023training/train.csv', index=False)

In [ ]:
dff =  pd.read_csv('/content/drive/MyDrive/dict_guid/DSC2023training/train.csv')
len(dff)

###PrepareDataVal

In [ ]:
with open('/content/drive/MyDrive/DataUITDSC/Bản sao của ise-dsc01-warmup.json', 'r') as file:
  data_val = json.load(file)

len(data_val)

In [ ]:
for i in data_val:
  print(data_val[i])
  break

In [ ]:
data_val_fr = {'key': [], 'context': [], 'claim': [], 'label': [], 'evidence': []}
for key in tqdm(data_val):
  data_val_fr['key'].append(key)
  data_val_fr['context'].append("".join(rdrsegmenter.word_segment(data_val[key]['context'])))
  data_val_fr['claim'].append(rdrsegmenter.word_segment(data_val[key]['claim'])[0].replace('\x00', ''))
  data_val_fr['label'].append(maplabel(data_val[key]['verdict']))
  data_val_fr['evidence'].append(data_val[key]['evidence'])


In [ ]:
val_df = pd.DataFrame(data_val_fr)

val_df.head()

In [ ]:
val_df = val_df.fillna(' ')

In [ ]:
val_df.head()

In [ ]:
val_df.to_csv('/content/drive/MyDrive/dict_guid/DSC2023training/val.csv', index=False)

In [ ]:
dff_val =  pd.read_csv('/content/drive/MyDrive/dict_guid/DSC2023training/val.csv')
len(dff_val)

In [ ]:
count2 = 0
for sample in val_df.values:
  if sample[3] == 0:
    count2 += 1
print(count2)

###PrepareData3


In [ ]:
ex = ' Ban tổ chức và Ban giám khảo sẽ chấm điểm, chọn bài thi vào vòng chung kết dựa trên 40% bình chọn của độc giả và 60% điểm Ban giám khảo'
if ex[0] == ' ':
  ex1 = ex[1:]
print(ex)
print(ex1)


In [ ]:
'Gueorguiev được phép trò chuyện với mọi người 30 phút mỗi ngày."Tôi muốn thử thách bản thân", Gueorguiev nói'.split('."')

In [ ]:
for j in data['46609']['context'].split('\n'):
      if not j:
        continue
      if j[0] == ' ':
        j = j[1:]
      j = j.replace('\n', '')
      print(j)

In [ ]:
print(data['46609']['evidence'])

In [ ]:
import re
count = 0
datafr_excep = {'key': [], 'context': [], 'claim': [], 'label': [], 'evidence': []}
evidence_df = {'key': [], 'sentence': [], 'claim': [], 'label': []}
for key in tqdm(data):
  if data[key]['verdict'] != "NEI":
    check = True
    for k in re.split('\. |\.\n|\.\s|\n', data[key]['context']):
      j = k
      if not j:
        continue
      if j[0] == ' ':
        j = j[1:]
      j = j.replace('\n', '')
      j = j.replace('. ', '')
      rd = np.random.randint(0, 8, 1)[0]
      if (j == data[key]['evidence'] or j+'.' == data[key]['evidence']):
        # print('split')
        # print(j)
        # print(data[key]['evidence'])
        s = j.replace('\x00', '')
        c = data[key]['claim'].replace('\x00', '')
        evidence_df['key'].append(key)
        evidence_df['sentence'].append("".join(rdrsegmenter.word_segment(s)))
        evidence_df['claim'].append("".join(rdrsegmenter.word_segment(c)))
        evidence_df['label'].append(1)
        count += 1
        check = False
      elif rd == 1:
        s = j.replace('\x00', '')
        c = data[key]['claim'].replace('\x00', '')
        evidence_df['key'].append(key)
        evidence_df['sentence'].append("".join(rdrsegmenter.word_segment(s)))
        evidence_df['claim'].append("".join(rdrsegmenter.word_segment(c)))
        evidence_df['label'].append(0)

    if check:
      datafr_excep['key'].append(key)
      datafr_excep['context'].append(data[key]['context'])
      datafr_excep['claim'].append(data[key]['claim'])
      datafr_excep['label'].append(data[key]['verdict'])
      datafr_excep['evidence'].append(data[key]['evidence'])


In [ ]:
count = 0
for i in evidence_df['label']:
  if i == 1:
    count += 1

print(count)


In [ ]:
evidence_train_df = pd.DataFrame(evidence_df)

In [ ]:
from sklearn.model_selection import train_test_split

evidence_train, evidence_test = train_test_split(evidence_train_df, test_size=0.1, random_state=128)

In [ ]:
evidence_train.head()

In [ ]:
evidence_test.head()

In [ ]:
evidence_train.to_csv('/content/drive/MyDrive/dict_guid/DSC2023evidence/evidence_train.csv', index=False)
evidence_test.to_csv('/content/drive/MyDrive/dict_guid/DSC2023evidence/evidence_test.csv', index=False)

In [ ]:
len(evidence_train.values)

In [ ]:
train_df.loc[586, 'claim'] = s

In [ ]:
train_df.values[586][2]

In [ ]:
train_df.to_csv('/content/drive/MyDrive/dict_guid/DSC2023/test.csv', index=False)

##FactClasify

In [ ]:
data_files = {"train": "train.csv", "validation": "val.csv"}
train_dataset = load_dataset("/content/drive/MyDrive/dict_guid/DSC2023training", split="train")
val_dataset = load_dataset("/content/drive/MyDrive/dict_guid/DSC2023training", split="validation")

In [ ]:
train_dataset[0]

In [ ]:
def tokenization(batched_text):
    a = tokenizer(batched_text['context'], padding = True, truncation=True, max_length=256, add_special_tokens = True)
    c = {'context_input_ids': a['input_ids'],'context_attention_mask': a['attention_mask']}
    return c



train_dataset = train_dataset.map(tokenization, batched = True, batch_size = len(train_dataset))
val_dataset = val_dataset.map(tokenization, batched = True, batch_size = len(val_dataset))

In [ ]:
train_dataset

In [ ]:
def tokenization2(batched_text):
    a = tokenizer(batched_text['claim'], padding = True, truncation=True, max_length=256, add_special_tokens = True)
    c = {'claim_input_ids': a['input_ids'],'claim_attention_mask': a['attention_mask']}
    return c



train_dataset = train_dataset.map(tokenization2, batched = True, batch_size = len(train_dataset))
val_dataset = val_dataset.map(tokenization2, batched = True, batch_size = len(val_dataset))

In [ ]:
train_dataset.set_format('torch', columns=['context_input_ids', 'context_attention_mask', 'claim_input_ids', 'claim_attention_mask', 'label'])
val_dataset.set_format('torch', columns=['context_input_ids', 'context_attention_mask', 'claim_input_ids', 'claim_attention_mask', 'label'])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
class ClaimClassifier(nn.Module):
    def __init__(self, n_classes):
        super(ClaimClassifier, self).__init__()
        self.bert_context = AutoModel.from_pretrained("vinai/phobert-base-v2")
        self.bert_claim = AutoModel.from_pretrained("vinai/phobert-base-v2")

        self.drop = nn.Dropout(p=0.3)

        self.classifier = nn.Sequential(nn.Linear(self.bert_claim.config.hidden_size, self.bert_claim.config.hidden_size),
                                 nn.ReLU(),
                                 nn.Linear(self.bert_claim.config.hidden_size, n_classes)
                                 )


    def forward(self, context_input_ids, context_attention_mask, claim_input_ids, claim_attention_mask):
        last_hidden_state_context, output_context = self.bert_context(
            input_ids=context_input_ids,
            attention_mask=context_attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        last_hidden_state_claim, output_claim = self.bert_claim(
            input_ids=claim_input_ids,
            attention_mask=claim_attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        output = output_context*output_claim

        x = self.drop(output)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
# Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
# Batchsize: 16, 32
optimizer = AdamW(model.parameters(), lr=2e-5)

# lr_scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=0,
#             num_training_steps=len(train_loader)*EPOCHS
#         )
best_acc = 0
for epoch in range(100):
    print(f'Epoch {24+epoch+1}/{2}')
    print('-'*30)

    model.train()
    losses = []
    correct = 0

    for data in tqdm(train_loader):
        context_input_ids = torch.tensor(data['context_input_ids']).to(device)
        context_attention_mask = torch.tensor(data['context_attention_mask']).to(device)
        claim_input_ids = torch.tensor(data['claim_input_ids']).to(device)
        claim_attention_mask = torch.tensor(data['claim_attention_mask']).to(device)
        targets = torch.tensor(data['label'], dtype=torch.long).to(device)

        optimizer.zero_grad()
        outputs = model(
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
            claim_input_ids=claim_input_ids,
            claim_attention_mask=claim_attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')
    if not (24+epoch+1)%2:
      stri = f'/content/drive/MyDrive/dict_guid/DSC2023model/factchecking_new_{24+epoch+1}.pth'
      torch.save(model.state_dict(), stri)


In [ ]:
model.eval()

In [ ]:
for data in dff.values:
  print(data[3])

In [ ]:
model.eval()
correct = 0
for data in tqdm(valid_loader):
        context_input_ids = torch.tensor(data['context_input_ids']).to(device)
        context_attention_mask = torch.tensor(data['context_attention_mask']).to(device)
        claim_input_ids = torch.tensor(data['claim_input_ids']).to(device)
        claim_attention_mask = torch.tensor(data['claim_attention_mask']).to(device)
        targets = torch.tensor(data['label'], dtype=torch.long).to(device)

        outputs = model(
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
            claim_input_ids=claim_input_ids,
            claim_attention_mask=claim_attention_mask
        )
        _, pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred == targets)

        # print(maptarget(pred), maptarget(targets))
print(f'Train Accuracy: {correct.double()/len(valid_loader.dataset)}')


##EvidenceExtract

In [ ]:
data_files = {"train": "evidence_train.csv", "test": "evidence_test.csv"}
evidence_train_dataset = load_dataset("/content/drive/MyDrive/dict_guid/DSC2023evidence", split="train")
evidence_val_dataset = load_dataset("/content/drive/MyDrive/dict_guid/DSC2023evidence", split="test")

In [ ]:
count = 0
for i in evidence_train_dataset:
  if i['label'] == 1:
    count += 1

print(count)

In [ ]:
print(len(evidence_train_dataset))

In [ ]:
def tokenization(batched_text):
    a = tokenizer(batched_text['sentence'], padding = True, truncation=True, max_length=256, add_special_tokens = True)
    c = {'sentence_input_ids': a['input_ids'],'sentence_attention_mask': a['attention_mask']}
    return c



evidence_train_dataset = evidence_train_dataset.map(tokenization, batched = True, batch_size = len(evidence_train_dataset))
evidence_val_dataset = evidence_val_dataset.map(tokenization, batched = True, batch_size = len(evidence_val_dataset))

In [ ]:
def tokenization2(batched_text):
    a = tokenizer(batched_text['claim'], padding = True, truncation=True, max_length=256, add_special_tokens = True)
    c = {'claim_input_ids': a['input_ids'],'claim_attention_mask': a['attention_mask']}
    return c



evidence_train_dataset = evidence_train_dataset.map(tokenization2, batched = True, batch_size = len(evidence_train_dataset))
evidence_val_dataset = evidence_val_dataset.map(tokenization2, batched = True, batch_size = len(evidence_val_dataset))

In [ ]:
evidence_train_dataset

In [ ]:
evidence_train_dataset.set_format('torch', columns=['sentence_input_ids', 'sentence_attention_mask', 'claim_input_ids', 'claim_attention_mask', 'label'])
evidence_val_dataset.set_format('torch', columns=['sentence_input_ids', 'sentence_attention_mask', 'claim_input_ids', 'claim_attention_mask', 'label'])

In [ ]:
evidence_train_loader = DataLoader(evidence_train_dataset, batch_size=32, shuffle=True)
evidence_valid_loader = DataLoader(evidence_val_dataset, batch_size=32, shuffle=False)

In [ ]:
class evidenceClassifier(nn.Module):
    def __init__(self, n_classes):
        super(evidenceClassifier, self).__init__()
        self.bert_sentence = AutoModel.from_pretrained("vinai/phobert-base-v2")
        self.bert_claim = AutoModel.from_pretrained("vinai/phobert-base-v2")

        self.drop = nn.Dropout(p=0.3)
        self.classifier = nn.Sequential(nn.Linear(self.bert_claim.config.hidden_size, self.bert_claim.config.hidden_size),
                                 nn.ReLU(),
                                 nn.Linear(self.bert_claim.config.hidden_size, n_classes)
                                 )

    def forward(self, sentence_input_ids, sentence_attention_mask, claim_input_ids, claim_attention_mask):
        last_hidden_state_sentence, output_sentence = self.bert_sentence(
            input_ids=sentence_input_ids,
            attention_mask=sentence_attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        last_hidden_state_claim, output_claim = self.bert_claim(
            input_ids=claim_input_ids,
            attention_mask=claim_attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        output = output_sentence*output_claim

        x = self.drop(output)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_evidence = evidenceClassifier(2).to(device)
model_evidence.load_state_dict(torch.load('/content/drive/MyDrive/dict_guid/DSC2023model/evidencechecking_new_31.pth'))

In [ ]:
criterion = nn.CrossEntropyLoss()
# Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
# Batchsize: 16, 32
optimizer = AdamW(model_evidence.parameters(), lr=2e-5)

# lr_scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=0,
#             num_training_steps=len(evidence_train_loader)*EPOCHS
#         )
best_acc = 0
for epoch in range(100):
    print(f'Epoch {1+epoch+1}/{100}')
    print('-'*30)

    model_evidence.train()
    losses = []
    correct = 0

    for data in tqdm(evidence_train_loader):
        sentence_input_ids = torch.tensor(data['sentence_input_ids']).to(device)
        sentence_attention_mask = torch.tensor(data['sentence_attention_mask']).to(device)
        claim_input_ids = torch.tensor(data['claim_input_ids']).to(device)
        claim_attention_mask = torch.tensor(data['claim_attention_mask']).to(device)
        targets = torch.tensor(data['label'], dtype=torch.long).to(device)

        optimizer.zero_grad()
        outputs = model_evidence(
            sentence_input_ids=sentence_input_ids,
            sentence_attention_mask=sentence_attention_mask,
            claim_input_ids=claim_input_ids,
            claim_attention_mask=claim_attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model_evidence.parameters(), max_norm=1.0)
        optimizer.step()
    print(f'Train Accuracy: {correct.double()/len(evidence_train_loader.dataset)} Loss: {np.mean(losses)}')
    torch.save(model_evidence.state_dict(), f'/content/drive/MyDrive/dict_guid/DSC2023model/evidencechecking_new_{1+epoch+1}.pth')


#Infer

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
with open('/content/drive/MyDrive/DataUITDSC/ise-dsc01-public-test-offcial.json', 'r') as file:
  data_test = json.load(file)

len(data_test)

In [ ]:
with open('/content/drive/MyDrive/DataUITDSC/ise-dsc01-public-test-offcial.json', 'r') as file:
  data_test_seg = json.load(file)

len(data_test_seg)

In [ ]:
for key in tqdm(data_test_seg):
  data_test_seg[key]['context'] = "".join(rdrsegmenter.word_segment(data_test_seg[key]['context']))
  data_test_seg[key]['claim'] = "".join(rdrsegmenter.word_segment(data_test_seg[key]['claim']))

In [ ]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = data_test_seg
        self.list_key = list(self.data)

    def __getitem__(self, index):
        context = self.data[self.list_key[index]]['context']
        claim = self.data[self.list_key[index]]['claim']

        return context, claim, self.list_key[index]

    def __len__(self):
        return len(self.data)



In [ ]:
dataset = MyDataset()
loader = DataLoader(
    dataset,
    batch_size=4,
    num_workers=4
)

In [ ]:
def sup_score(stri1, stri2):
  num_wo = 0
  le = len(stri2.split())
  for word in stri2.split():
    if word in stri1:
      num_wo += 1
  return num_wo/le


In [ ]:
import re

In [ ]:
print(list(re.split('\n\.|\s\.|\n', data_test_seg['41942']['context'])))

In [ ]:
for k in re.split('\n\.|\s\.|\n', data_test_seg['41942']['context']):
  print(k)

In [ ]:
ans = {}
model.eval()
model_evidence.eval()

correct = 0
for dat in tqdm(loader):
        check = True
        contexts, claims, keys = dat
        a = tokenizer(contexts, padding = True, truncation=True, max_length=256, add_special_tokens = True)
        b = tokenizer(claims, padding = True, truncation=True, max_length=256, add_special_tokens = True)

        context_input_ids = torch.tensor(a['input_ids']).to(device)
        context_attention_mask = torch.tensor(a['attention_mask']).to(device)
        claim_input_ids = torch.tensor(b['input_ids']).to(device)
        claim_attention_mask = torch.tensor(b['attention_mask']).to(device)
        # targets = torch.tensor(data['label'], dtype=torch.long).to(device)

        outputs = model(
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
            claim_input_ids=claim_input_ids,
            claim_attention_mask=claim_attention_mask
        )
        _, preds = torch.max(outputs, dim=1)
        # correct += torch.sum(pred == targets)
        for idx, key_id in enumerate(keys):
          if preds[idx] != 2:
            ss = []
            cc = []
            for k in re.split('\. |\.\n|\.\s|\n', data_test[key_id]['context']):
              sen = tokenizer("".join(rdrsegmenter.word_segment(k)), padding = True, truncation=True, max_length=256, add_special_tokens = True)
              cla = tokenizer(contexts[idx], padding = True, truncation=True, max_length=256, add_special_tokens = True)
              print(contexts[idx])
              sentence_input_ids = torch.tensor([sen['input_ids']]).to(device)
              sentence_attention_mask = torch.tensor([sen['attention_mask']]).to(device)
              claim_input_ids = torch.tensor([b['input_ids']]).to(device)
              claim_attention_mask = torch.tensor([b['attention_mask']]).to(device)
              print(sentence_input_ids.shape)
              print(sentence_attention_mask.shape)
              print(claim_input_ids.shape)
              print(claim_attention_mask.shape)

              out_evi = model_evidence(
                  sentence_input_ids=sentence_input_ids,
                  sentence_attention_mask=sentence_attention_mask,
                  claim_input_ids=claim_input_ids,
                  claim_attention_mask=claim_attention_mask
              )

              sof = nn.Softmax()
              out_prob = sof(out_evi)

              sup = sup_score(k, data_test[key_id]['claim'])
              # print(out_prob)
              # break

              score = out_prob[0][1].item()

              score = 0.3*score + 0.7*sup

              ss.append(k)
              cc.append(score)
            # break
            idx = torch.argmax(torch.tensor(cc))
            evi = ss[idx]
            print(cc[idx])
            print(evi)
            print(data_test[key_id]['claim'])
            ans[key_id] = {"context": data_test[key_id]['context'],
                  "claim": data_test[key_id]['claim'],
                  "verdict": maptarget(preds[idx]),
                  "evidence": evi}
            check = False
              # if preds[idx]_evi == 1:
              #   evi = k
              #   print(evi)
              #   print(data_test[key_id]['claim'])
              #   ans[key_id] = {"context": data_test[key_id]['context'],
              #         "claim": data_test[key_id]['claim'],
              #         "verdict": maptarget(preds[idx]),
              #         "evidence": evi}
              #   check = False
              #   break
          if check:
            ans[key_id] = {"context": data_test[key_id]['context'],
                        "claim": data_test[key_id]['claim'],
                        "verdict": maptarget(preds[idx]),
                        "evidence": ""}
          print(maptarget(preds[idx]))
        # print(maptarget(preds[idx]), maptarget(targets))
# print(f'Train Accuracy: {correct.double()/len(valid_loader.dataset)}')


In [ ]:
ans = {}
model.eval()
correct = 0
for key in tqdm(data_test):
        check = True
        a = tokenizer("".join(rdrsegmenter.word_segment(data_test[key]['context'])), padding = True, truncation=True, max_length=256, add_special_tokens = True)
        b = tokenizer("".join(rdrsegmenter.word_segment(data_test[key]['claim'])), padding = True, truncation=True, max_length=256, add_special_tokens = True)

        context_input_ids = torch.tensor([a['input_ids']]).to(device)
        context_attention_mask = torch.tensor([a['attention_mask']]).to(device)
        claim_input_ids = torch.tensor([b['input_ids']]).to(device)
        claim_attention_mask = torch.tensor([b['attention_mask']]).to(device)
        # targets = torch.tensor(data['label'], dtype=torch.long).to(device)

        outputs = model(
            context_input_ids=context_input_ids,
            context_attention_mask=context_attention_mask,
            claim_input_ids=claim_input_ids,
            claim_attention_mask=claim_attention_mask
        )
        _, pred = torch.max(outputs, dim=1)
        # correct += torch.sum(pred == targets)
        if pred != 2:
          ss = []
          cc = []
          for k in re.split('\. |\.\n|\.\s|\n', data_test[key]['context']):
            sen = tokenizer("".join(rdrsegmenter.word_segment(k)), padding = True, truncation=True, max_length=256, add_special_tokens = True)
            cla = tokenizer("".join(rdrsegmenter.word_segment(data_test[key]['claim'])), padding = True, truncation=True, max_length=256, add_special_tokens = True)

            sentence_input_ids = torch.tensor([sen['input_ids']]).to(device)
            sentence_attention_mask = torch.tensor([sen['attention_mask']]).to(device)
            claim_input_ids = torch.tensor([b['input_ids']]).to(device)
            claim_attention_mask = torch.tensor([b['attention_mask']]).to(device)


            out_evi = model_evidence(
                sentence_input_ids=sentence_input_ids,
                sentence_attention_mask=sentence_attention_mask,
                claim_input_ids=claim_input_ids,
                claim_attention_mask=claim_attention_mask
            )

            sof = nn.Softmax()
            out_prob = sof(out_evi)

            sup = sup_score(k, data_test[key]['claim'])
            # print(out_prob)
            # break

            score = out_prob[0][1].item()

            score = 0.3*score + 0.7*sup

            ss.append(k)
            cc.append(score)
          # break
          idx = torch.argmax(torch.tensor(cc))
          evi = ss[idx]
          print(cc[idx])
          print(evi)
          print(data_test[key]['claim'])
          ans[key] = {"context": data_test[key]['context'],
                "claim": data_test[key]['claim'],
                "verdict": maptarget(pred),
                "evidence": evi}
          check = False
            # if pred_evi == 1:
            #   evi = k
            #   print(evi)
            #   print(data_test[key]['claim'])
            #   ans[key] = {"context": data_test[key]['context'],
            #         "claim": data_test[key]['claim'],
            #         "verdict": maptarget(pred),
            #         "evidence": evi}
            #   check = False
            #   break
        if check:
          ans[key] = {"context": data_test[key]['context'],
                      "claim": data_test[key]['claim'],
                      "verdict": maptarget(pred),
                      "evidence": ""}
        print(maptarget(pred))
        # print(maptarget(pred), maptarget(targets))
# print(f'Train Accuracy: {correct.double()/len(valid_loader.dataset)}')


In [ ]:
print(len(ans))

In [ ]:
# json_obj = json.dumps(ans)
with open("/content/drive/MyDrive/dict_guid/ResultDSC/ans2810_2.json", "w", encoding='utf-8') as outfile:
    # outfile.write(json_obj)
    json.dump(ans, outfile, ensure_ascii=False)